In [1]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('./data/311_calls_small_almost_cleaned.csv')
show_progress = lambda: df['Incident Zip'].isna().sum()

place_desc = ['Borough', 'Incident Zip',
       'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Facility Type']

show_progress()

/var/folders/b3/cjvms6c10k77tgv6k_rw3jbh0000gn/T/ipykernel_46729/3730440412.py:5: DtypeWarning: Columns (31,32,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/311_calls_small_almost_cleaned.csv')


89526

In [2]:
df[place_desc].head()

,Borough,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type
0,BROOKLYN,11226.0,2709 CLARENDON ROAD,CLARENDON ROAD,NaN,NaN,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
1,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BROOKLYN,11211.0,201 SOUTH 4 STREET,SOUTH 4 STREET,SOUTH 5 PLACE,ROEBLING STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
3,MANHATTAN,10032.0,533 WEST 158 STREET,WEST 158 STREET,AMSTERDAM AVENUE,BROADWAY,NaN,NaN,ADDRESS,NEW YORK,NaN,NaN
4,STATEN ISLAND,10303.0,6 MACORMAC PLACE,MACORMAC PLACE,NORTHFIELD AVENUE,HOLLAND AVENUE,NaN,NaN,ADDRESS,STATEN ISLAND,NaN,NaN


In [3]:
bu = df['Borough'] == 'Unspecified'
df.loc[bu, 'Borough'] = np.nan

In [11]:
df['Borough'].unique()

array(['BROOKLYN', nan, 'MANHATTAN', 'STATEN ISLAND', 'BRONX', 'QUEENS'],
      dtype=object)

In [5]:
df['Borough'].isna().sum()

18600

In [10]:
for col in place_desc:
    print(f'{col:<22}: {(df[col].isna().sum() / df.shape[0]):.2%}')

Borough               : 1.86%
Incident Zip          : 8.95%
Incident Address      : 14.45%
Street Name           : 14.45%
Cross Street 1        : 46.77%
Cross Street 2        : 46.97%
Intersection Street 1 : 87.54%
Intersection Street 2 : 87.56%
Address Type          : 8.37%
City                  : 8.91%
Landmark              : 98.91%
Facility Type         : 72.09%


In [12]:
is_brooklyn = lambda: df['Borough'] == 'BROOKLYN'
is_staten = lambda: df['Borough'] == 'STATEN ISLAND'
is_bronx = lambda: df['Borough'] == 'BRONX'
is_manhattan = lambda: df['Borough'] == 'MANHATTAN'
is_queens = lambda: df['Borough'] == 'QUEENS'

In [15]:
df.loc[is_brooklyn(), place_desc].sort_values(by='Street Name').head()

,Borough,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type
396356,BROOKLYN,NaN,481-483 .DEKALB AVE,.DEKALB AVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
455159,BROOKLYN,NaN,4821 1 AVENUE,1 AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
551777,BROOKLYN,11232.0,4000 1 AVENUE,1 AVENUE,40 STREET,41 STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,Precinct
170575,BROOKLYN,11220.0,5401 1 AVENUE,1 AVENUE,54 STREET,55 STREET,54 STREET,55 STREET,NaN,BROOKLYN,1 AVENUE,NaN
669247,BROOKLYN,11220.0,5601 1 AVENUE,1 AVENUE,56 STREET,57 STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN


In [23]:
def uniform_names(x):
    pt1 = re.compile(r'\ ST\.*$')
    pt2 = re.compile(r'\ AVE\.*$')
    pt3 = re.compile(r'\ RD\.*$')
    pt4 = re.compile(r' \ PL\.*$')
    pt5 = re.compile(r'\w{3}')
    try:
        if re.search(pt1, x):
            return re.sub(pt1, ' STREET', x).upper()
        elif re.search(pt2, x):
            return re.sub(pt2, ' AVENUE', x).upper()
        elif re.search(pt3, x):
            return re.sub(pt3, ' ROAD', x).upper()
        elif re.search(pt4, x):
            return re.sub(pt4, ' PLACE', x).upper()
        elif not re.search(pt5, x):
            return np.nan
    except:
        return x
    return x


In [24]:
street_cols = ['Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2']
for col in street_cols:
    df[col] = df[col].apply(lambda x: uniform_names(x))
    

In [27]:
df[place_desc].head()

,Borough,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type
0,BROOKLYN,11226.0,2709 CLARENDON ROAD,CLARENDON ROAD,NaN,NaN,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BROOKLYN,11211.0,201 SOUTH 4 STREET,SOUTH 4 STREET,SOUTH 5 PLACE,ROEBLING STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
3,MANHATTAN,10032.0,533 WEST 158 STREET,WEST 158 STREET,AMSTERDAM AVENUE,BROADWAY,NaN,NaN,ADDRESS,NEW YORK,NaN,NaN
4,STATEN ISLAND,10303.0,6 MACORMAC PLACE,MACORMAC PLACE,NORTHFIELD AVENUE,HOLLAND AVENUE,NaN,NaN,ADDRESS,STATEN ISLAND,NaN,NaN


In [28]:
df.dropna(subset=place_desc, how='all', inplace=True)

In [29]:
show_progress()

75241

In [16]:
# SANDBOX: DONT USE

In [20]:
df.loc[df['Street Name'].str.contains('\ RD', na=False), place_desc].head()

,Borough,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type
1308,NaN,11430.0,21A NORTH SERVICE RD,NORTH SERVICE RD,NaN,NaN,NaN,NaN,NaN,JAMAICA,NaN,NaN
1490,QUEENS,11103.0,441 NEWTOWN RD,NEWTOWN RD,31 AVE,44 ST,NEWTOWN ROAD,44 STREET,INTERSECTION,Astoria,NaN,NaN
5560,NaN,NaN,3 BREWSTER RD,BREWSTER RD,NaN,NaN,NaN,NaN,NaN,NEWARK,NaN,NaN
9561,QUEENS,11101.0,ED KOCH QUEENSBORO BR UPPER RDWY,ED KOCH QUEENSBORO BR UPPER RDWY,ED KOCH QUEENSBORO BRIDGE,VERNON BOULEVARD,NaN,NaN,ADDRESS,LONG ISLAND CITY,NaN,NaN
17325,NaN,NaN,200 LIBERTY CORNER RD,LIBERTY CORNER RD,NaN,NaN,NaN,NaN,NaN,WARREN,NaN,NaN


In [35]:
tmp = df.loc[is_brooklyn(), place_desc].copy()
tmp.shape

(322885, 12)

In [50]:
tmp = tmp.drop_duplicates(subset=street_cols, keep='first')

In [47]:
tmp.head()

,Borough,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type
0,BROOKLYN,11226.0,2709 CLARENDON ROAD,CLARENDON ROAD,NaN,NaN,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
2,BROOKLYN,11211.0,201 SOUTH 4 STREET,SOUTH 4 STREET,SOUTH 5 PLACE,ROEBLING STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
8,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,AVE Y,INTERSECTION,NaN,NaN,NaN
16,BROOKLYN,11225.0,410 EASTERN PARKWAY,EASTERN PARKWAY,BEDFORD AVENUE,ROGERS AVENUE,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
24,BROOKLYN,11228.0,NaN,NaN,NaN,NaN,12 AVENUE,86 STREET,INTERSECTION,BROOKLYN,NaN,NaN


In [51]:
tmp = tmp.dropna(subset=['Street Name', 'Cross Street 1'], how='all')

In [59]:
tmp.groupby(['Street Name', 'Cross Street 1'])['Incident Zip'].unique()

Street Name            Cross Street 1                 
1 AVENUE               40 STREET                               [11232.0]
                       42 STREET                               [11232.0]
                       44 STREET                          [nan, 11232.0]
                       47 STREET                               [11232.0]
                       48 STREET                               [11232.0]
                                                               ...      
YORK STREET            GOLD STREET                             [11201.0]
                       JAY STREET                              [11201.0]
                       MELQUAIN JATELLE ANDERSON DRIVE         [11201.0]
                       NAVY STREET                             [11201.0]
ZENITA THOMPSON PLACE  MONTGOMERY STREET                       [11225.0]
Name: Incident Zip, Length: 21610, dtype: object

In [57]:
length = (tmp.groupby(['Street Name', 'Cross Street 1'])['Incident Zip']
 .unique()
 .apply(lambda x: len(x))
)

In [58]:
length[length > 2]

Street Name       Cross Street 1 
4 AVENUE          4 AVENUE           4
75 STREET         PITKIN AVENUE      3
ATLANTIC AVENUE   ATLANTIC AVENUE    5
AVENUE P          AVENUE P           3
BEDFORD AVENUE    BEDFORD AVENUE     3
                  NORTH 12 STREET    3
BROADWAY          BROADWAY           3
EASTERN PARKWAY   EASTERN PKWY       3
FULTON STREET     FULTON STREET      4
KINGS HIGHWAY     BEND               3
MC DONALD AVENUE  18 AVENUE          4
Name: Incident Zip, dtype: int64

In [39]:
a = pd.DataFrame({'a': [1,1,3], 'b': [2,2,4]})
a

,a,b
0,1,2
1,1,2
2,3,4


In [43]:
a.drop_duplicates(subset=['a', 'b'], keep='first')

,a,b
0,1,2
2,3,4
